In [12]:
import pandas as pd
import numpy as np
from maxijdefs import *
from maxij_nicerdefs import *
import glob
from scipy.signal import fftconvolve
import matplotlib
import matplotlib.pyplot as plt
from pylab import subplot, subplots_adjust, legend
%matplotlib inline

In [2]:
# info for reading x-ray data (directory and filter for night)
# for example, for 3/28 the NICER date is 113
# filename_filter = 'ni1200120113_0mpu7_silver_GTI*.lc' # for 3/28
xpathnam = 'delivery/'
filename_filter = '*_0mpu7_silver_GTI*.lc'

# info to read rho optical data (directory and night)
opathnam = './maxij_data_copies/'
onights = ['2018-03-28'] # put other nights here
night = onights[0] # to pick out night. add functionality to loop over all nights later?


In [3]:
## time calibration stuff:

# computer clock is ~51.5s fast (per lab tests + SSE calculation)
# (((POSSIBLY 8.5 s slow, but this seems to work?)))
clocklag = -51.5
# clocklag = 8.5
# clocklag -=60

# do not change this! it is absolutely calibrated to the 28th.
# unix epoch timestamp - nicer absolute time
# both at 2018-03-28 08:05:21.000 UTC
dt = 1522224321-133689923
# print dt

In [4]:
## read in nicer xray data for night and make pandas df
# get filenames for night's xray data
ftest = xpathnam + filename_filter
flistD = glob.glob(ftest)

# all files
flist = [fnam.split('/')[1] for fnam in flistD]
flist = natsorted(flist)
# print len(flist)
ndat_t = []
ndat_cts = []
ndat_cts1 = []
ndat_cts2 = []
ndat_cts34 = []
ndat_fnam = []

for fileX in flist:
    t,cts=rd_nicer_lc(xpathnam,[fileX])
    t1,cts1=rd_nicer_lc1(xpathnam,[fileX])
    t2,cts2=rd_nicer_lc2(xpathnam,[fileX])
    t34,cts34=rd_nicer_lc34(xpathnam,[fileX])
    filename_column = [fileX for x in np.arange(len(t))]

    ndat_t.append(t)
    ndat_cts.append(cts)
    ndat_cts1.append(cts1)
    ndat_cts2.append(cts2)
    ndat_cts34.append(cts34)
    ndat_fnam.append(filename_column)

ndict = {'filename': [item for sublist in ndat_fnam for item in sublist],
          'time': [item for sublist in ndat_t for item in sublist],
          'cts': [item for sublist in ndat_cts for item in sublist],
          'cts1': [item for sublist in ndat_cts1 for item in sublist],
          'cts2': [item for sublist in ndat_cts2 for item in sublist],
          'cts34': [item for sublist in ndat_cts34 for item in sublist]
          }

nicerdata = pd.DataFrame(ndict, index=ndict['time'])
# print nicerdata

In [19]:
#read in optical data as pandas dataframe:
rhodata = pd.read_pickle(opathnam+'data_'+night+'.pkl')
# print data
print len(rhodata)

11729


In [6]:
# fix/calibrate time for both optical and x-ray data so that they're all using unix epoch time
rhodata['fixed_time'] = rhodata['os_time'] + clocklag
nicerdata['unix_time'] = nicerdata['time'] + dt

In [7]:
## clean up data
# test = np.abs(rhodata['gauss_params_3_sigma_x'] - rhodata['gauss_params_4_sigma_y'])
# n, bins, patches = plt.hist(test, 50, normed=1, facecolor='green', alpha=0.75)

shiftgood = rhodata['shift_corr_amplitude'] < 10.
gaussgood = rhodata['gauss_sigma_avg'] < 3.
gaussnonzero = rhodata['gauss_sigma_avg'] > 0.
gaussgood2 = np.abs(rhodata['gauss_params_3_sigma_x'] - rhodata['gauss_params_4_sigma_y']) < 1.
rhoclean = rhodata[shiftgood & gaussgood & gaussgood2 & gaussnonzero]
# plt.plot(rhodata['fixed_time'],rhodata['phot_maxij']-np.median(rhodata['phot_maxij']),'.-m')
# plt.plot(rhodata['fixed_time'],rhodata['phot_tyc']-np.median(rhodata['phot_tyc']),'.-c')
# plt.plot(rhoclean['fixed_time'],rhoclean['phot_maxij']-np.median(rhoclean['phot_maxij']),'.-r')
# plt.plot(rhoclean['fixed_time'],rhoclean['phot_tyc']-np.median(rhoclean['phot_tyc']),'.-g')
# plt.show()

In [8]:
## pull out just datasets we are going to use
xt = nicerdata['unix_time']
xlc = nicerdata['cts']
ot = rhoclean['fixed_time']
olc = rhoclean['phot_maxij']
tlc = rhoclean['phot_tyc']

In [9]:
## define timeframe based on optical data
## make 1-s timeseries' for whole night (with zeroes where no data)
optical_time_seconds = np.arange(np.floor(np.min(rhoclean['fixed_time'])),np.ceil(np.max(rhoclean['fixed_time'])))
xts = make_1s_ts(optical_time_seconds, np.array(xt),np.array(xlc))
ots = make_1s_ts(optical_time_seconds, np.array(ot),np.array(olc))
tts = make_1s_ts(optical_time_seconds, np.array(ot),np.array(tlc))

# plt.plot(optical_time_seconds,xts,'b.')
# plt.plot(optical_time_seconds,ots,'r.')
# plt.show()

In [10]:
## split into 64-s chunks
ntot = 64
xts_c = chunks(xts,ntot)
ots_c = chunks(ots,ntot)
tts_c = chunks(tts,ntot)
time_c = chunks(optical_time_seconds,ntot)
tc = np.arange(ntot)
ctot = len(ots_c)

datadict = {'chunkID':np.arange(ctot),
            'chunk_length':ntot,
            
           }


In [22]:
# plot of time series over chunk of data
# print ctot
# fig2,ax1=plt.subplots(figsize=[6,100],facecolor='w')
# subplots_adjust(hspace=0.000)
# for c,v in enumerate(xrange(ctot)):
#     v = v+1
#     ax1 = subplot(ctot,1,v)
#     ax1.plot(tc, xts_c[c],'.-b')
#     ax1.plot(tc, ots_c[c],'.-r')
#     ax1.plot(tc, tts_c[c],'.-k',alpha=.3)
#     ax1.plot([0,ntot],[0,0],'k')
#     ax1.set_xlim(0,ntot)
# ax1.legend(['MaxiJ@Nicer','MaxiJ@RHO',"Tycho@RHO"])